In [ ]:
import requests
import pandas as pd
from datetime import datetime

# 오늘 날짜
today = datetime.today().strftime('%Y%m%d')

# 요청 파라미터 구성
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
payload = {
    "bld": "dbms/MDC/STAT/standard/MDCSTAT03901",
    "mktId": "ALL",  # STK: 코스피, KSQ: 코스닥, ALL: 전체
    "trdDd": today   # 조회 날짜
}

# POST 요청
response = requests.post(url, data=payload, headers=headers)
response.raise_for_status()

# 결과 파싱
data = response.json()['OutBlock1']
df = pd.DataFrame(data)

# 주요 컬럼 정리
df = df[["idxNm", "tddClspr", "flucRt", "mktcap", "lstCnt", "secugrpNm"]]
df.columns = ["섹터명", "종가", "등락률", "시가총액", "종목수", "시장구분"]

# 시가총액 숫자형으로 변환
df["시가총액"] = df["시가총액"].str.replace(",", "").astype(float)

# 정렬
df_sorted = df.sort_values(by="시가총액", ascending=False)

# 결과 저장
df_sorted.to_excel("krx_섹터_시총_정리.xlsx", index=False)
print("✅ 저장 완료: krx_섹터_시총_정리.xlsx")


섹터별 추출:   0%|          | 0/33 [00:00<?, ?it/s]

[수산·농림업] 섹터 파싱 오류: 'NoneType' object has no attribute 'find_all'


섹터별 추출:   3%|▎         | 1/33 [00:01<00:51,  1.61s/it]

[광업] 섹터 파싱 오류: 'NoneType' object has no attribute 'find_all'


섹터별 추출:   6%|▌         | 2/33 [00:03<00:48,  1.58s/it]

...

[부동산업] 섹터 파싱 오류: 'NoneType' object has no attribute 'find_all'


섹터별 추출:  97%|█████████▋| 32/33 [00:50<00:01,  1.56s/it]

[서비스업] 섹터 파싱 오류: 'NoneType' object has no attribute 'find_all'


섹터별 추출: 100%|██████████| 33/33 [00:51<00:00,  1.57s/it]

✅ 저장 완료: krx_sector_top10.xlsx
